In [359]:
import warnings
warnings.filterwarnings(action='ignore')

from helper.regrassion import *
from helper.util import *
from helper.plot import *
from helper.analysis import *


### [2] 데이터 가져오기

중고차 가격 예측을 위한 데이터 세트

https://www.kaggle.com/datasets/rishabhkarn/used-car-dataset

| 변수명 | 설명 |
|:--:|--:|
| car_name | Name of the car |
| registration_year | Vehicle registration year |
| insurance_validity | Type of insurance |
| fuel_type | Type of fuel |
| seats | Number of seats in the car |
| kms_driven | Total kms driven |
| ownsership | Number of ownership |
| transmission | Transmission mode |
| manufacturing_year | manufacturing year |
| mileage(kmpl) | Mileage of the car |
| engine(cc) | Engine capacity |
| max_power(bhp) | max power |
| torque(Nm) | torque |
| price(in lakhs) | Price of the used car |

In [360]:
origin = my_read_data("./res/Used_Car_Dataset.csv",info= False).drop('Unnamed: 0',axis=1)
df = origin.drop_duplicates().drop('engine(cc)',axis=1)
del_num = []
for i,v in enumerate(df['price(in lakhs)']):
    if v>100:
        del_num.append(i)
for i in list(reversed(del_num)):
    df = df.drop(i)

In [361]:
A = []
A_ = []
for i in origin['registration_year'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
# A, A_

In [362]:
origin['registration_year'].unique()

map_registration_year = {a:None for a in A}
for i in A_:
    map_registration_year[i] = i
map_registration_year['Third Party insurance'] = None
df['registration_year'] = df['registration_year'].map(map_registration_year)
# df['registration_year']

In [363]:
month_dict = {'Apr':4,
 'Aug':8,
 'Feb':2,
 'Dec':12,
 'Jan':1,
 'Jul':7,
 'Jun':6,
 'Mar':3,
 'May':5,
 'Nov':11,
 'Oct':10,
 'Sep':9}

In [364]:
month = []
year = []
for i in df['registration_year']:
    if i:
        try:
            month.append(month_dict[i[:3]])
            year.append(int(i[-2:]))
        except : print(i)
    else:
        month.append(None)
        year.append(None)
    

In [365]:
df['month']=month
df['year']=year

In [366]:
origin['insurance_validity'].value_counts()
map_insurance_validity = {a:a for a in origin['insurance_validity'].unique()}
map_insurance_validity['Petrol'] = None
map_insurance_validity['Not Available'] = None
map_insurance_validity['Third Party'] = 'Third Party insurance'
df['insurance_validity'] = df['insurance_validity'].map(map_insurance_validity)
df['insurance_validity']

0               Comprehensive
1               Comprehensive
2               Comprehensive
3               Comprehensive
4               Comprehensive
                ...          
1548            Comprehensive
1549    Third Party insurance
1550            Comprehensive
1551            Comprehensive
1552            Comprehensive
Name: insurance_validity, Length: 1131, dtype: object

In [367]:
origin['fuel_type'].value_counts()
map_fuel_type = {a:a for a in origin['fuel_type'].unique()}
map_fuel_type['CNG'] = None
map_fuel_type['5 Seats'] = None
df['fuel_type'] = df['fuel_type'].map(map_fuel_type)
df['fuel_type']

0       Petrol
1       Petrol
2       Diesel
3       Petrol
4       Petrol
         ...  
1548    Diesel
1549    Petrol
1550    Petrol
1551    Petrol
1552    Petrol
Name: fuel_type, Length: 1131, dtype: object

In [368]:
origin['seats'].value_counts()
map_seats = {a:a if a<10 else None for a in origin['seats'].unique()}
df['seats'] = df['seats'].map(map_seats)
df['seats']

0       5.0
1       5.0
2       5.0
3       5.0
4       5.0
       ... 
1548    5.0
1549    5.0
1550    5.0
1551    5.0
1552    5.0
Name: seats, Length: 1131, dtype: float64

In [369]:
origin['ownsership'].value_counts()
map_ownsership = {a:a if "Owner" in a else None for a in origin['ownsership'].unique()}
df['ownsership'] = df['ownsership'].map(map_ownsership)
df['ownsership']


0        First Owner
1        First Owner
2        First Owner
3        First Owner
4        First Owner
            ...     
1548     First Owner
1549            None
1550     First Owner
1551    Second Owner
1552     First Owner
Name: ownsership, Length: 1131, dtype: object

In [370]:
A = []
A_ = []
for i in origin['transmission'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
origin['transmission'].value_counts()

Manual                 835
Automatic              668
2017                    28
2014                     5
2011                     3
2023                     2
2020                     2
2021                     2
2022                     2
Power Windows Front      2
2018                     2
2015                     1
2016                     1
Name: transmission, dtype: int64

In [371]:
map_transmission = {a:None for a in A}
for i in A_:
    map_transmission[i] = i
map_transmission['Power Windows Front']=None
df['transmission'] = df['transmission'].map(map_transmission)
df['transmission']

0       Automatic
1       Automatic
2       Automatic
3          Manual
4       Automatic
          ...    
1548    Automatic
1549         None
1550       Manual
1551       Manual
1552       Manual
Name: transmission, Length: 1131, dtype: object

In [372]:
origin['manufacturing_year'].value_counts()
A = []
A_ = []
for i in origin['manufacturing_year'].unique():
    try : 
        int(i)
        A.append(i)
    except:A_.append(i)
A, A_

(['2017',
  '2020',
  '2018',
  '2019',
  '2015',
  '2021',
  '2014',
  '2022',
  '2010',
  '2011',
  '2023',
  '2016',
  '2009',
  '2013',
  '2012',
  '2007'],
 ['Power Steering', 'Power Windows Front', 'Air Conditioner'])

In [373]:
map_manufacturing_year = {a:a for a in A}
for i in A_:
    map_manufacturing_year[i] = None
df['manufacturing_year'] = df['manufacturing_year'].map(map_manufacturing_year)
df['manufacturing_year']

0       2017
1       2020
2       2018
3       2019
4       2019
        ... 
1548    2020
1549    None
1550    2017
1551    2018
1552    2018
Name: manufacturing_year, Length: 1131, dtype: object

In [374]:
for i,v in enumerate(origin['mileage(kmpl)']):
    if v>40 : df['mileage(kmpl)'][i] = None


In [375]:
# max = 13519
# min = 352

# for i,v in enumerate(origin['engine(cc)']):
#     if v < min or v > max : df['engine(cc)'][i] = None


In [376]:
df.dtypes

car_name               object
registration_year      object
insurance_validity     object
fuel_type              object
seats                 float64
kms_driven              int64
ownsership             object
transmission           object
manufacturing_year     object
mileage(kmpl)         float64
max_power(bhp)        float64
torque(Nm)            float64
price(in lakhs)       float64
month                 float64
year                  float64
dtype: object

In [377]:
df2 = my_set_category(df.dropna().drop('car_name',axis=1))
df2['price'] = df2['price(in lakhs)']
df2.drop('price(in lakhs)',axis=1,inplace=True)

df2.dtypes

registration_year     category
insurance_validity    category
fuel_type             category
seats                  float64
kms_driven               int64
ownsership            category
transmission          category
manufacturing_year    category
mileage(kmpl)          float64
max_power(bhp)         float64
torque(Nm)             float64
month                  float64
year                   float64
price                  float64
dtype: object

In [378]:
df2.head()

,registration_year,insurance_validity,fuel_type,seats,kms_driven,ownsership,transmission,manufacturing_year,mileage(kmpl),max_power(bhp),torque(Nm),month,year,price
0,Jul-17,Comprehensive,Petrol,5.0,56000,First Owner,Automatic,2017,7.81,2996.0,333.0,7.0,17.0,63.75
1,Jan-21,Comprehensive,Petrol,5.0,30615,First Owner,Automatic,2020,17.40,999.0,9863.0,1.0,21.0,8.99
2,Sep-18,Comprehensive,Diesel,5.0,24000,First Owner,Automatic,2018,20.68,1995.0,188.0,9.0,18.0,23.75
3,Dec-19,Comprehensive,Petrol,5.0,18378,First Owner,Manual,2019,16.50,1353.0,13808.0,12.0,19.0,13.56
4,Aug-19,Comprehensive,Petrol,5.0,44900,First Owner,Automatic,2019,14.67,1798.0,17746.0,8.0,19.0,24.00


In [379]:
import numpy as np
a = (np.array(df2['year']) - int(df2['year'].min())) *12 + df2['month']
df2['registration'] = a
df2['mileage'] = df2['mileage(kmpl)']
# df2['engine'] = df2['engine(cc)']
df2['max_power'] = df2['max_power(bhp)']
df2['torque'] = df2['torque(Nm)']
df2.drop(['torque(Nm)','max_power(bhp)','mileage(kmpl)'],axis=1,inplace=True)
df2['kms_driven2']=df2['kms_driven']**2
df2['registration2'] = df2['registration']**2
df2['torque2'] = df2['torque']**2
df2['max_power2'] = df2['max_power']**2
# df2['engine2'] = df2['engine']**2
df2['mileage'] = df2['mileage'] **2

In [380]:

df3 =df2.drop(['year','month','manufacturing_year','registration_year'],axis=1)
df3.head()


,insurance_validity,fuel_type,seats,kms_driven,ownsership,transmission,price,registration,mileage,max_power,torque,kms_driven2,registration2,torque2,max_power2
0,Comprehensive,Petrol,5.0,56000,First Owner,Automatic,63.75,103.0,60.9961,2996.0,333.0,3136000000,10609.0,110889.0,8976016.0
1,Comprehensive,Petrol,5.0,30615,First Owner,Automatic,8.99,145.0,302.7600,999.0,9863.0,937278225,21025.0,97278769.0,998001.0
2,Comprehensive,Diesel,5.0,24000,First Owner,Automatic,23.75,117.0,427.6624,1995.0,188.0,576000000,13689.0,35344.0,3980025.0
3,Comprehensive,Petrol,5.0,18378,First Owner,Manual,13.56,132.0,272.2500,1353.0,13808.0,337750884,17424.0,190660864.0,1830609.0
4,Comprehensive,Petrol,5.0,44900,First Owner,Automatic,24.00,128.0,215.2089,1798.0,17746.0,2016010000,16384.0,314920516.0,3232804.0


In [381]:
categories = [x for x in df3.columns if df3[x].dtypes == 'category']
categories.append('seats')
df3['seats']=df3['seats'].astype('category')

In [382]:
for c in categories:
    tmp = df3[[c, 'price']]
    my_anova(tmp, hue=c, target='price')

pingouin.anova
+----+--------------------+-------------+------+-------------+----------+----------+-------------+
|    | Source             |          SS |   DF |          MS |        F |    p-unc |         np2 |
|----+--------------------+-------------+------+-------------+----------+----------+-------------|
|  0 | insurance_validity | 2.93146e+06 |    2 | 1.46573e+06 | 0.146578 | 0.863679 | 0.000325262 |
|  1 | Within             | 9.00967e+09 |  901 | 9.99964e+06 |      nan |      nan |         nan |
+----+--------------------+-------------+------+-------------+----------+----------+-------------+

statsmodels.anova.anova_lm
+-----------------------+------+-------------+-------------+----------+----------+
|                       |   df |      sum_sq |     mean_sq |        F |   PR(>F) |
|-----------------------+------+-------------+-------------+----------+----------|
| C(insurance_validity) |    2 | 2.93146e+06 | 1.46573e+06 | 0.146578 | 0.863679 |
| Residual              |  901 

> insurance_validity, fuel_type, transmission 은 제외한다.

In [383]:
df4=my_standard_scaler(df3.drop(['insurance_validity','seats','transmission','fuel_type'],axis=1),'price')
df4

,kms_driven,registration,mileage,max_power,torque,kms_driven2,registration2,torque2,max_power2,ownsership,price
0,0.063479,-0.210436,-2.128652,2.424085,-0.161620,-0.057280,-0.371122,-0.089686,2.323741,First Owner,63.75
1,-0.498196,1.041891,-0.335804,-0.994633,-0.057205,-0.130101,1.099571,-0.088667,-0.696404,First Owner,8.99
2,-0.644561,0.207006,0.590434,0.710446,-0.163208,-0.142066,0.063760,-0.089686,0.432466,First Owner,23.75
3,-0.768955,0.654266,-0.562057,-0.388611,-0.013982,-0.149957,0.591126,-0.087689,-0.381214,First Owner,13.56
4,-0.182122,0.534997,-0.985057,0.373196,0.029165,-0.094374,0.444282,-0.086386,0.149599,First Owner,24.00
...,...,...,...,...,...,...,...,...,...,...,...
1544,-0.528354,0.296458,-0.956646,0.018827,-0.151978,-0.132803,0.164150,-0.089671,-0.115968,First Owner,11.05
1546,-0.745324,0.087737,-0.124605,-0.652248,-0.155550,-0.148619,-0.066139,-0.089679,-0.529990,First Owner,6.50
1550,-0.091404,-0.240254,-0.309923,-0.652248,-0.155550,-0.081623,-0.400067,-0.089679,-0.529990,First Owner,5.85
1551,-0.290541,0.087737,0.034450,-0.994633,-0.164446,-0.108152,-0.066139,-0.089687,-0.696404,Second Owner,4.75


In [384]:
dummy_df = my_dummies(df4)
my_pretty_table(dummy_df.head())


+----+--------------+----------------+-----------+-------------+------------+---------------+-----------------+------------+--------------+---------+---------------------------+--------------------------+
|    |   kms_driven |   registration |   mileage |   max_power |     torque |   kms_driven2 |   registration2 |    torque2 |   max_power2 |   price |   ownsership_Second Owner |   ownsership_Third Owner |
|----+--------------+----------------+-----------+-------------+------------+---------------+-----------------+------------+--------------+---------+---------------------------+--------------------------|
|  0 |    0.0634794 |      -0.210436 |  -2.12865 |     2.42408 |   -0.16162 |    -0.0572803 |       -0.371122 | -0.0896857 |      2.32374 |   63.75 |                         0 |                        0 |
|  1 |    -0.498196 |        1.04189 | -0.335804 |   -0.994633 | -0.0572048 |     -0.130101 |         1.09957 | -0.0886674 |    -0.696404 |    8.99 |                         0 |   

In [385]:
x_train, x_test, y_train, y_test = my_train_test_split(dummy_df, "price", test_size=0.3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((632, 11), (272, 11), (632,), (272,))

In [386]:
ignore = ['torque','registration2','max_power','kms_driven2','torque2','kms_driven','mileage','registration']
x_train2 = x_train.drop(ignore,axis=1)
x_test2 = x_test.drop(ignore,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test, degree = 2,use_plot=False, resid_test=False)

[회귀식]
price = 13.485 * max_power2 + 0.417 * ownsership_Second Owner + -9.915 * ownsership_Third Owner + 14.385

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |    -0.00413347 |             358.128 |         3.31852e+07 |          5760.66 |                         129.126 |              -107.081 |
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+

[선형회귀분석 결과보고]
+----+------------+-------------------------+--------------------+------------+------------------+--------+------------+---------+
|    | 종속변수   | 독립변수    

LinearRegression(n_jobs=-1)